<a href="https://colab.research.google.com/github/aliakbarbadri/persian-poetry-creator/blob/master/char-gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

# Load the data

In [0]:
url = "https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/shahname2.txt"
filepath = keras.utils.get_file("shahname.txt", url) 
corpus = open(filepath, 'rb').read().decode(encoding='utf-8')

In [6]:
print(len(corpus),"chars")

2554745 chars


In [8]:
print(corpus[:110])

چو از بلخ بامی به جیحون رسید	سپهدار لشکر فرود آورید
بشد شهریار از میان سپاه	فرود آمد از باره بر شد به گاه
بخوا


In [10]:
vocab = sorted(set(corpus))
print ('{} unique characters'.format(len(vocab)))

48 unique characters


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in corpus])

In [13]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '(' :   3,
  ')' :   4,
  '«' :   5,
  '»' :   6,
  '،' :   7,
  '؟' :   8,
  'ء' :   9,
  'آ' :  10,
  'أ' :  11,
  'ؤ' :  12,
  'ئ' :  13,
  'ا' :  14,
  'ب' :  15,
  'ت' :  16,
  'ث' :  17,
  'ج' :  18,
  'ح' :  19,
  ...
}


In [18]:
seq_length = 1000
examples_per_epoch = len(corpus)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

چ
و
 
ا
ز
 
ب
ل
خ
 


In [19]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'چو از بلخ بامی به جیحون رسید\tسپهدار لشکر فرود آورید\nبشد شهریار از میان سپاه\tفرود آمد از باره بر شد به گاه\nبخواند او گرانمایه جاماسپ را\tکجا رهنمون بود گشتاسپ را\nسر موبدان بودو شاه ردان\tچراغ بزرگان و اسپهبدان\nچنان پاک تن بود و تابنده جان\tکه بودی بر او آشکارا نهان\nستاره\u200cشناس و گرانمایه بود\tابا او به دانش کرا پایه بود\nبپرسید ازو شاه و گفتا خدای\tترا دین به داد و پاکیزه رای\nچو تو نیست اندر جهان هیچ کس\tجهاندار دانش ترا داد و بس\nببایدت کردن ز اختر شمار\tبگویی همی مر مرا روی کار\nکه چون باشد آغاز و فرجام جنگ\tکرا بیشتر باشد اینجا درنگ\nنیامد خوش آن پیر جاماسپ را\tبه روی دژم گفت گشتاسپ را\nکه میخواستم کایزد دادگر\tندادی مرا این خرد وین هنر\nمرا گر نبودی خرد شهریار\tنکردی زمن بودنی خواستار\nمگر با من از داد پیمان کند\tکه نه بد کند خود نه فرمان کند\nجهانجوی گفتا به نام خدای\tبدین و به دین آور پاک رای\nبه جان زریر آن نبرده سوار\tبه جان گرانمایه اسفندیار\nکه نه هرگزت روی دشمن کنم\tنفرمایمت بد نه خود من کنم\nتو هرچ اندرین کار دانی بگوی\tکه تو چاره\u200cدانی و من چاره\u200cجوی\nخ

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [26]:
for input_example, target_example in  dataset.take(1):
  print (repr(''.join(idx2char[input_example.numpy()])))
  print (repr(''.join(idx2char[target_example.numpy()])))

'چو از بلخ بامی به جیحون رسید\tسپهدار لشکر فرود آورید\nبشد شهریار از میان سپاه\tفرود آمد از باره بر شد به گاه\nبخواند او گرانمایه جاماسپ را\tکجا رهنمون بود گشتاسپ را\nسر موبدان بودو شاه ردان\tچراغ بزرگان و اسپهبدان\nچنان پاک تن بود و تابنده جان\tکه بودی بر او آشکارا نهان\nستاره\u200cشناس و گرانمایه بود\tابا او به دانش کرا پایه بود\nبپرسید ازو شاه و گفتا خدای\tترا دین به داد و پاکیزه رای\nچو تو نیست اندر جهان هیچ کس\tجهاندار دانش ترا داد و بس\nببایدت کردن ز اختر شمار\tبگویی همی مر مرا روی کار\nکه چون باشد آغاز و فرجام جنگ\tکرا بیشتر باشد اینجا درنگ\nنیامد خوش آن پیر جاماسپ را\tبه روی دژم گفت گشتاسپ را\nکه میخواستم کایزد دادگر\tندادی مرا این خرد وین هنر\nمرا گر نبودی خرد شهریار\tنکردی زمن بودنی خواستار\nمگر با من از داد پیمان کند\tکه نه بد کند خود نه فرمان کند\nجهانجوی گفتا به نام خدای\tبدین و به دین آور پاک رای\nبه جان زریر آن نبرده سوار\tبه جان گرانمایه اسفندیار\nکه نه هرگزت روی دشمن کنم\tنفرمایمت بد نه خود من کنم\nتو هرچ اندرین کار دانی بگوی\tکه تو چاره\u200cدانی و من چاره\u200cجوی\nخ

In [27]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 42 ('چ')
  expected output: 39 ('و')
Step    1
  input: 39 ('و')
  expected output: 2 (' ')
Step    2
  input: 2 (' ')
  expected output: 14 ('ا')
Step    3
  input: 14 ('ا')
  expected output: 24 ('ز')
Step    4
  input: 24 ('ز')
  expected output: 2 (' ')


In [28]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
dataset

<BatchDataset shapes: ((64, 1000), (64, 1000)), types: (tf.int64, tf.int64)>

# Model

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [32]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 1000, 48) # (batch_size, sequence_length, vocab_size)


In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           12288     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 48)            49200     
Total params: 3,999,792
Trainable params: 3,999,792
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [37]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 1000, 48)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.8722224


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [40]:
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
39/39 [==============================] - 20s 512ms/step - loss: 3.3189
Epoch 2/10
39/39 [==============================] - 21s 526ms/step - loss: 2.5066
Epoch 3/10
39/39 [==============================] - 20s 521ms/step - loss: 2.3542
Epoch 4/10
39/39 [==============================] - 20s 513ms/step - loss: 2.2371
Epoch 5/10
39/39 [==============================] - 20s 516ms/step - loss: 2.0891
Epoch 6/10
39/39 [==============================] - 20s 519ms/step - loss: 1.9561
Epoch 7/10
39/39 [==============================] - 20s 519ms/step - loss: 1.8415
Epoch 8/10
39/39 [==============================] - 20s 521ms/step - loss: 1.7358
Epoch 9/10
39/39 [==============================] - 20s 519ms/step - loss: 1.6526
Epoch 10/10
39/39 [==============================] - 20s 519ms/step - loss: 1.5856


In [41]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
# model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

# model.build(tf.TensorShape([1, None]))

# model.summary()

In [0]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 1.0
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [45]:
print(generate_text(model, start_string=u"که ایران چو باغی ست خرم بهار"))

که ایران چو باغی ست خرم بهارت و تیمار داد
سوی چیبهٔ روزگاره شد از از بیدم به کمن
چنین گنج لشکر به پادا همی‌رانجوی
من او را جهاندیده بر نیست پیلان‌فزوش
بقاسان سوم چرد و نیکوید و درفشاه
پدی هر چو یاعت ز هشمان نامدرد	چنین گفت داود شه تیخ تن بی‌ندار
به بهرام بر دو فراوار و کیورده‌اوی
ز گردان لشکر ز مرد آندای بیشت
چنین داد ساخنده رس مر ز کرده‌هم
اگر هم همی کشک مادر سوی پیش اوی	هدی گفت ایدر شبوتن بکوید همی
بسی کستن سان کسی ناکو آمد پدیم
به جای جزان آربودنمگاران سپس	که کا پرستی دست بش کرد کست
سپیدی بدو گفت مهتار برداشته
شداور تو زن گشت ما را و و ز یادر گرفت
ز در او جساند در آفرین تو هم نو توست
که شد روی هسته مرا خواستی پیر دست
ستاور ز له چون برو بنده بریان مت
برآت بفتن و زیر دودان مراست	که از رافتی گند و جوهی شتند	بفرمودر تاکیان بر همی شویمند
هره ناموردر بدر یزدگار آنل بیاه	بمودان که پاید بر آن رخش هر	ز راسپ سراینده شاهنگهان	که بشنید بازور و دینار چو شاندیشهی
بروییم بخشانگ آگان بسیار هیشیزد	ستانش که گون سر کهان رنج راه
که هنگانه بد پند کا گردنش راه شاه
چه گورد نوادش نرا شاه جان بد کمان	نماید 

# Use model

In [0]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [0]:
import warnings
warnings.filterwarnings('ignore')

X_new = preprocess(["که"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

' '

In [0]:
tf.random.set_seed(42)
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]])

In [0]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [0]:
tf.random.set_seed(42)
next_char("که ایران چو با", temperature=1)

'ش'

In [0]:
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [0]:
tf.random.set_seed(42)
print(complete_text("که ایران چو باغی ست خرم بهار", temperature=0.2))

که ایران چو باغی ست خرم بهار

چو بر تیر و بر تیره شد از درد و گاه
به پیش آمد از خواب و به داد	به نزدیک شاه با راه بود	به دیدار گ
